In [1]:
import numpy as np
import torch
from transformers import AutoTokenizer, EvalPrediction, AdapterType, BertTokenizer, BertTokenizerFast, AutoModelWithHeads
from transformers import (Trainer,TrainingArguments, set_seed)
import warnings

warnings.simplefilter("ignore", UserWarning)
warnings.simplefilter("ignore", FutureWarning)
warnings.simplefilter("ignore", DeprecationWarning)

import tensorflow as tf
import torch
import torch.optim as optim
from torch.utils.data import TensorDataset, DataLoader, RandomSampler, Dataset
from keras.preprocessing.sequence import pad_sequences
from sklearn.model_selection import train_test_split
from tqdm import tqdm, trange
import pandas as pd
from sklearn.metrics import f1_score, accuracy_score
from statistics import mode

from transformers import (
    Trainer,
    TrainingArguments,
    )
import os
import re
import emoji
import random

import nltk
nltk.download('stopwords')
from nltk import word_tokenize, pos_tag
from nltk.stem import WordNetLemmatizer
from nltk.tokenize import sent_tokenize, TweetTokenizer
from nltk.corpus import wordnet, stopwords
from imblearn.over_sampling import SMOTE

# specify GPU device
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
n_gpu = torch.cuda.device_count()
torch.cuda.get_device_name(0)

Using TensorFlow backend.
[nltk_data] Downloading package stopwords to C:\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


'GeForce MX150'

In [2]:
def preprocess(df):
    
    #removes URL
    pattern = r'https.?://[^\s]+[\s]?'
    df["tweet"] = df["tweet"].str.replace(pat=pattern, repl="", regex=True)
    
    #removes usernames/mentions
    pattern = r'@[^\s]+'
    df["tweet"] = df["tweet"].str.replace(pat=pattern, repl="", regex=True)
    
    #removes emoji and smiley
    pattern = re.compile("["
                         u"\U0001F600-\U0001F64F"
                         u"\U0001F300-\U0001F5FF"
                         u"\U0001F680-\U0001F6FF"
                         u"\U0001F1E0-\U0001F1FF"
                         u"\U00002500-\U00002BEF"
                         u"\U00002702-\U000027B0"
                         u"\U00002702-\U000027B0"
                         u"\U000024C2-\U0001F251"
                         u"\U0001f926-\U0001f937"
                         u"\U00010000-\U0010ffff"
                         u"\u2640-\u2642"
                         u"\u2600-\u2B55"
                         u"\u200d"
                         u"\u23cf"
                         u"\u23e9"
                         u"\u231a"
                         u"\ufe0f"
                         u"\u3030"
                         "]+", flags=re.UNICODE)
    df["tweet"] = df["tweet"].str.replace(pat=pattern, repl="", regex=True)
    
    #removes numbers
    pattern = r'\d+'
    df["tweet"] = df["tweet"].str.replace(pat=pattern, repl="", regex=True)
    
    #removes punctuation
    pattern = r"[^\w\s]"
    df["tweet"] = df["tweet"].str.replace(pat=pattern, repl=" ", regex=True)

    #removes stop words
    stop_words = stopwords.words("english")    
    remove_stop_words = lambda row: " ".join([token for token in row.split(" ")
                                              if token not in stop_words])
    df["tweet"] = df["tweet"].apply(remove_stop_words)
    
    #removes extra spaces
    pattern = r"[\s]+"
    df["tweet"] = df["tweet"].str.replace(pat=pattern, repl=" ", regex=True)
    
    return(df)

In [3]:
def train_validate_test_split(df,seed= 2018, train_percent=.8, validate_percent=.125):
    train_tweet, test_tweet, train_label, test_label  = train_test_split(df.tweet.tolist(), df.label.tolist(), train_size=train_percent, stratify=df['label'])
    train_tweet, validate_tweet, train_label, validate_label = train_test_split(train_tweet, train_label, test_size=validate_percent, stratify=train_label)
    return train_tweet, validate_tweet, test_tweet, train_label, validate_label, test_label

In [4]:
class Dataset(torch.utils.data.Dataset):
    def __init__(self, encodings, labels):
        self.encodings = encodings
        self.labels = labels

    def __getitem__(self, idx):
        item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
        item['labels'] = torch.tensor(self.labels[idx])
        return item

    def __len__(self):
        return len(self.labels)

In [5]:
def model_initialize(model_name, num_labels = 2):
    model = AutoModelWithHeads.from_pretrained(model_name)
    model.add_adapter("sst-2", AdapterType.text_task)
    model.add_classification_head("sst-2", num_labels=num_labels)
    model.train_adapter(["sst-2"])
    model.set_active_adapters([["sst-2"]])

    # we freeze the model weights which in turn they wont be updated
    for param in model.base_model.parameters():
        param.requires_grad = False
    
    training_args = TrainingArguments(
      logging_steps=50,
      per_device_train_batch_size=16,
      per_device_eval_batch_size=16,
      evaluation_strategy='epoch',
      output_dir="./",
      overwrite_output_dir=True,
      learning_rate=1e-4,
      warmup_steps=500,
      weight_decay=0.01,
      do_train=True,
      do_eval=True,
      do_predict=True,
      num_train_epochs=5)
    
    set_seed(training_args.seed)
    return model, training_args

In [6]:
def Train_model(model, training_args, train_dataset, val_dataset, test_dataset, test_label):
    trainer = Trainer(
        model=model,
        args=training_args,
        train_dataset = train_dataset,
        eval_dataset = val_dataset)
    
    trainer.train()
    pred = trainer.predict(test_dataset)
    pred = np.argmax(pred[0],axis=1)
    print("The Weighted F1- Score is: ", f1_score(np.array(test_label), pred, average = 'weighted'))
    print("The Macro F1- Score is: ", f1_score(np.array(test_label), pred, average = 'macro'))
    print("Accuracy is: ", accuracy_score(np.array(test_label), pred))

In [7]:
directory = r"C:\Users\samsu\Desktop\Fall20\CS 695-002\Project\Datasets\Monolingual/"
languages = ['French', 'Arabic', 'English']
model_name = "bert-base-multilingual-uncased"
tokenizer = BertTokenizerFast.from_pretrained(model_name)
for lang in languages:
    df = pd.read_csv(os.path.join(directory, lang+'.csv'))
    df = preprocess(df)
    train_tweet, validate_tweet, test_tweet, train_label, validate_label, test_label = train_validate_test_split(df,seed= 2018, train_percent=.8, validate_percent=.125)
    
    train_encodings = tokenizer(train_tweet, truncation=True, padding='max_length', max_length=128)
    val_encodings = tokenizer(validate_tweet, truncation=True, padding='max_length', max_length=128)        
    test_encodings = tokenizer(test_tweet, truncation=True, padding='max_length', max_length=128)
    train_dataset = Dataset(train_encodings, train_label)
    val_dataset = Dataset(val_encodings, validate_label)
    test_dataset = Dataset(test_encodings, test_label)
    
    model, training_args = model_initialize(model_name, num_labels = 2)
    Train_model(model, training_args, train_dataset, val_dataset, test_dataset, test_label)
    # model.save_adapter(os.path.join("./", lang), "sst-2", with_head=True)
    print("="*200)

Some weights of the model checkpoint at bert-base-multilingual-uncased were not used when initializing BertModelWithHeads: ['cls.predictions.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.decoder.weight', 'cls.seq_relationship.weight', 'cls.seq_relationship.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.LayerNorm.bias']
- This IS expected if you are initializing BertModelWithHeads from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModelWithHeads from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


RuntimeError: Expected object of device type cuda but got device type cpu for argument #1 'self' in call to _th_addmm